# bike share problem
>datetime - hourly date + timestamp  
>season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
>holiday - whether the day is considered a holiday
>workingday - whether the day is neither a weekend nor holiday
>weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
>2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
>3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
>4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
>temp - temperature in Celsius
>atemp - "feels like" temperature in Celsius
>humidity - relative humidity
>windspeed - wind speed
>casual - number of non-registered user rentals initiated
>registered - number of registered user rentals initiated
>count - number of total rentals

## load libs

In [203]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from scipy.stats import norm
from scipy import stats

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor

%run /Users/yaoyao/Documents/datascience/toolbox/toolbox1_dataview.ipynb
%matplotlib inline

## read in train and test data

In [26]:
with open('train.csv') as f:
    df_train = pd.read_csv(f)
with open('test.csv') as f:
    df_test = pd.read_csv(f)

## EDA

In [9]:
prf = pandas_profiling.ProfileReport(df_train)
prf.to_file('EDA.html')

In [10]:
# observations:
# col didn't exist in test, need to be removed: casual, registered

## Feature Selection


In [27]:
def initialprep(df_train,df_test):
    df_train = df_train.drop(columns=['casual','registered'])
    df_train_X = df_train.iloc[:,:9]
    df_train_Y = df_train.iloc[:,9]
    df_train_X['trainset'] = 1
    df_test['trainset'] = 0
    df_concat_X = pd.concat([df_train_X,df_test])
    return df_concat_X, df_train_Y

In [45]:
def handledatetime(df_concat_X):
    df_concat_X['year'] = pd.to_datetime(df_concat_X['datetime']).dt.year
    df_concat_X['month'] = pd.to_datetime(df_concat_X['datetime']).dt.month
    df_concat_X['day'] = pd.to_datetime(df_concat_X['datetime']).dt.day
    df_concat_X['hour'] = pd.to_datetime(df_concat_X['datetime']).dt.hour
    df_concat_X.drop(columns='datetime', inplace=True)
    return df_concat_X

In [210]:
def FeatureSelection(df_train,df_test):
    df_concat_X,df_train_Y = initialprep(df_train,df_test)
    #remove atemp since it's highly correlated with temp
    df_concat_X.drop(columns = 'atemp',inplace = True)
    #handle date time
    df_concat_X = handledatetime(df_concat_X)
    # check again for non-correlated values
    droplist = ['day','weather','year','month']
    df_concat_X = df_concat_X.drop(columns=droplist)
    #counts is skewed, apply np.log1p
    Y_train = np.log1p(df_train_Y)
    #windspeed is skewed, apply np.log1p
    df_concat_X['windspeed'] = np.log1p(df_concat_X['windspeed'])
    #add polynominal features
    
    X_train = df_concat_X[df_concat_X['trainset'] == 1].drop(columns='trainset')
    X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
    X_test = df_concat_X[df_concat_X['trainset'] != 1].drop(columns='trainset')
    X_test = PolynomialFeatures(degree=2).fit_transform(X_test)
    return X_train,X_test,Y_train
X_train,X_test,Y_train = FeatureSelection(df_train,df_test)

#X_train.head()
#Y_train.head()

In [49]:
prf = pandas_profiling.ProfileReport(df_concat_X[df_concat_X['trainset'] == 1])
prf.to_file('EDA2.html')

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


## model learning


In [155]:
def rms_ev(n_fold,model):
    kf = KFold(n_fold,shuffle=True,random_state=1).get_n_splits(X_train)
    rmes = np.sqrt(abs(cross_val_score(model,X_train,Y_train,scoring='neg_mean_squared_error',cv=kf)))
    return rmes

In [211]:
#LN_pipe = make_pipeline(RobustScaler(),Lasso(alpha = 0.001))
#LN_pipe = make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005, l1_ratio=.9))
GBGR = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.02,
                                   max_depth=3, max_features='sqrt',
                                   min_samples_leaf=20, min_samples_split=10, 
                                   loss='huber', random_state =5)
#parameters = {'learning_rate':[50**(-x) for x in range(0,3)],'max_depth': [3,4,5],'min_samples_leaf':[10,15,20]}
#GBGR_gs = GridSearchCV(GradientBoostingRegressor(n_estimators=1000,max_features='sqrt',min_samples_split=10,loss='huber',random_state =5),param_grid=parameters,scoring='neg_mean_squared_error',cv=5);
GBGR.fit(X_train,Y_train)
#print("The best value of Alpha is: ",GBGR_gs.best_params_)
#print("The best score achieved with Alpha=14 is: ",GBGR_gs.best_score_)
#X_train_scaled = RobustScaler().fit_transform(X_train)
rms_ev(5,GBGR).mean()

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.02, loss='huber', max_depth=3,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=20, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)

0.5542559512139404

In [213]:
y_predict = GBGR.predict(X_test)

In [216]:
df_test['datetime']

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,trainset
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,0
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,0
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,0
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,0


In [219]:
np.expm1(y_predict)
output = pd.DataFrame({'datetime':df_test['datetime'],
                       'count':np.expm1(y_predict).astype(int)})
output.to_csv('submit1.csv',index = False)

array([15.17328902,  7.35581214,  4.26913206, ..., 82.51843221,
       67.80718871, 40.58069549])

In [179]:
lngr = Lasso(alpha = 0.001).fit(X_train,Y_train)
X_train_scaled = RobustScaler().fit_transform(X_train)
for i in range(0,len(lngr.predict(X_train_scaled))):
    print(np.expm1(lngr.predict(X_train_scaled))[i], np.expm1(Y_train)[i])

14.123660112797454 16.0
14.207200469297232 40.00000000000001
14.331363236972276 32.0
14.54524009886418 12.999999999999996
14.672162870661982 1.0
14.80012193330498 1.0
14.83823497804915 2.0000000000000004
14.868241039224172 3.0
15.190301822685235 8.000000000000002
15.52022717783257 14.0
15.812718665815812 35.99999999999999
15.853274295796783 56.00000000000001
16.187138073167763 84.00000000000001
16.48170160392491 94.0
16.624435071481603 105.99999999999996
16.66694872329426 109.99999999999997
16.709564926512574 93.0
16.910300406182337 67.00000000000001
16.94423206399621 35.0
17.090741970363744 36.99999999999999
17.190671949104203 35.99999999999999
17.339193971920277 33.99999999999999
17.422195657116983 28.000000000000004
17.80655897663859 39.0
14.59857014928251 16.999999999999996
14.628122157494982 16.999999999999996
14.6577301529666 9.000000000000002
14.934310574441536 5.999999999999999
15.064410003715897 3.0
15.368713708311118 2.0000000000000004
15.459131515140703 1.0
15.63642859159926

16.708365990987975 128.99999999999997
16.927599461270653 127.99999999999997
16.998714726679864 82.99999999999999
17.004890766472855 84.00000000000001
17.265496772912385 73.99999999999997
17.41462972061457 41.00000000000001
17.564980299370916 56.999999999999986
17.628998560965105 26.0
17.75201759122222 43.99999999999999
14.368405033007349 39.0
14.493883993286843 23.000000000000004
14.62038745607255 16.0
14.544615251966118 14.999999999999998
14.876501416626349 1.0
15.006128849424712 2.0000000000000004
15.136814655909628 1.0
15.217572079645858 3.0
15.307155007521112 17.999999999999996
15.474302935030334 32.0
15.608811259035152 78.99999999999997
15.857915935398868 93.0
16.02191414036584 103.99999999999997
16.26829236746054 118.00000000000003
16.455007301510495 91.00000000000001
16.52424788665679 113.00000000000003
16.611948783173133 99.00000000000004
16.709233995154143 104.99999999999997
16.77030135726564 67.00000000000001
16.794773447431755 61.00000000000001
16.940063063745043 56.99999999

17.270063651710057 78.0
17.200231350846764 36.99999999999999
17.40652846686701 25.000000000000004
14.171825592856289 10.000000000000002
14.295699534026154 3.9999999999999996
14.324677743818405 2.0000000000000004
14.498380855693535 1.0
14.575943208405466 3.0
14.800757050805094 2.0000000000000004
15.020538111851677 39.0
15.14235180736519 100.00000000000003
15.349984264153044 243.00000000000006
15.517571190462453 135.00000000000006
15.643164124580528 47.99999999999999
15.77905115551367 50.0
16.04831280000608 64.99999999999997
16.24086004703252 50.0
16.282448439398998 63.99999999999998
16.441564770993967 51.00000000000001
16.58397053193086 82.99999999999999
16.755031985462818 175.99999999999994
16.843887851510413 151.99999999999997
16.803252144915554 100.99999999999996
16.948610987550154 56.00000000000001
17.103778468104128 54.000000000000014
17.19437965180429 29.000000000000004
17.34293194703177 12.0
14.142853445378666 16.999999999999996
14.26649083663104 6.999999999999998
14.327050006867

18.785484551375845 77.0
15.274132306878823 29.000000000000004
15.338688592889593 31.0
15.428940546425231 16.999999999999996
15.502633225215186 6.999999999999998
15.576656455503095 3.0
15.712000468917598 3.0
15.831052252604579 5.999999999999999
15.915284020005615 22.0
16.053392837876046 45.0
16.255601844652702 55.00000000000002
16.45119171530453 105.99999999999996
16.64899857020566 123.00000000000003
16.77472527563389 118.99999999999997
16.938373909731343 155.0
17.15107649659832 196.0
17.23249388600185 170.99999999999994
17.381357373865697 120.00000000000003
17.444742442195462 126.99999999999997
17.364015464917845 87.99999999999999
17.41782447513529 58.999999999999986
17.5195611983311 47.00000000000001
17.69006740009759 33.00000000000001
17.764207226042796 43.99999999999999
17.80947023556979 29.000000000000004
14.35377028787873 6.999999999999998
14.382240194731187 3.0
14.499200530914317 3.9999999999999996
14.544615251966118 2.0000000000000004
14.671532922054345 1.0
14.790692923189699 2.

15.955190335408215 43.99999999999999
16.173968315367016 121.99999999999994
16.422547401762863 180.99999999999997
16.647355832154474 194.0
16.791441822648938 220.99999999999997
16.90892993736082 249.9999999999999
17.130646433463 304.00000000000006
17.202567146324856 256.00000000000006
17.293662722463512 241.00000000000009
17.30865936486488 146.99999999999994
17.276403549117223 100.99999999999996
17.2442045609957 77.0
17.3449822392636 58.000000000000014
17.49476416476496 36.99999999999999
17.44311689839315 28.000000000000004
14.297123363628904 11.0
14.422020327940606 1.0
14.54793704291546 1.0
14.67488183454777 3.0
14.840982753788543 8.000000000000002
15.061968952360482 28.000000000000004
15.277627526364427 87.99999999999999
15.504705043254024 241.00000000000009
15.717670274416204 140.00000000000003
16.00350429323561 84.00000000000001
16.214011420492273 71.0
16.436129047665013 94.0
16.58757301146198 75.0
16.749498569004096 78.99999999999997
16.866709705950782 100.99999999999996
17.0879035

...

20.60579335117262 35.0
20.681168836156402 61.99999999999999
20.744697547382966 86.0
21.036656178303286 185.00000000000009
21.17072278905424 335.99999999999994
21.351740873770282 392.0
21.593421043085225 320.0
21.836796429839463 314.0
21.832835160758457 305.9999999999999
21.8879125121733 333.0000000000001
22.074786261194575 157.00000000000003
22.263185784925742 105.99999999999996
22.453123540913563 114.00000000000001
22.644612088417343 116.00000000000003
22.701647607654987 113.00000000000003
22.895165288398825 78.99999999999997
22.94043547111962 80.00000000000003
19.094846498932768 51.00000000000001
19.018039466769867 36.99999999999999
19.244127434266083 14.999999999999998
19.409415377729424 10.000000000000002
19.44808169169808 6.999999999999998
19.797697234482783 18.999999999999996
19.96750492887825 58.999999999999986
20.13869905816363 154.00000000000003
20.43358990049489 397.0
20.732594553645434 332.0
20.887411695409494 298.0000000000001
21.204214003583445 308.0000000000001
21.4442990

21.020376129117572 387.00000000000006
21.373747880232315 218.00000000000003
21.83963886436573 268.0
22.20709054340832 377.0000000000001
22.470139718885047 332.0
22.69846336214981 284.99999999999994
22.694352628384127 352.99999999999994
22.937219145178776 450.0000000000001
22.9578052133623 889.9999999999997
23.07770314047253 787.9999999999997
23.224187072907295 513.0000000000002
23.421971150109584 387.00000000000006
23.430351252130027 283.00000000000006
23.553600998834018 229.00000000000006
23.754074653571394 117.00000000000004
19.596129916023564 56.00000000000001
19.6458118883754 16.0
19.81437947911207 10.000000000000002
20.104745971407034 5.0
20.277060637487693 5.999999999999999
20.328385150521456 35.99999999999999
20.43598565287517 130.99999999999997
20.61100480330178 154.00000000000003
20.787452938777736 467.00000000000017
20.96534172658783 389.00000000000006
21.35334071579187 224.0
21.66517615346285 252.00000000000003
22.1610510888006 332.0
22.228916052052508 375.00000000000006
22.

19.88214128730814 26.999999999999996
19.82382718803813 130.99999999999997
19.99384822658982 340.9999999999999
20.454019744077527 693.0000000000001
20.742075543362365 327.0000000000001
21.02262015969468 197.0
21.37692737105879 251.00000000000003
21.772196668916045 262.00000000000006
22.06603325157273 298.99999999999994
22.25436130929657 274.0000000000001
22.468459946888 355.9999999999999
22.635642925506275 434.00000000000006
22.60619286384608 653.9999999999999
22.798931181866124 496.0000000000001
22.365655251529397 361.00000000000017
22.69161276385249 254.99999999999994
22.6875032183911 198.99999999999991
22.942664012589766 163.00000000000006
22.938510919796663 162.0
18.906470499599163 121.99999999999994
18.954488875916276 76.00000000000001
19.180670259405563 43.99999999999999
19.281665342646672 20.0
19.256394344231484 5.999999999999999
19.48599778462893 10.000000000000002
19.460472187553805 16.0
19.756620718296045 71.0
20.24448864015201 145.99999999999997
20.664471634678357 258.9999999

22.246294609271384 380.0000000000001
22.43609445327975 619.0000000000002
22.55338145513501 579.9999999999998
22.74568858197798 405.99999999999983
22.81526661092099 258.99999999999994
23.10913574066705 202.99999999999994
23.004582325641167 113.00000000000003
23.20057338819423 73.99999999999997
18.668994259791447 107.99999999999997
18.82958639307797 69.00000000000003
18.991489718636597 50.0
19.15471494200526 14.999999999999998
19.319272856129775 5.0
19.357768392103196 12.999999999999996
19.523984183915392 26.999999999999996
19.691557083683414 62.99999999999998
19.860498171834596 143.99999999999997
20.031663995079438 193.00000000000003
20.314048967903233 256.00000000000006
20.76911574509112 392.0
20.89024428044219 446.00000000000017
21.19561939160371 508.9999999999999
21.50525457210582 479.0
21.93830975471003 484.00000000000006
22.005522056198238 443.0
21.929430995399706 320.0
22.11664373184983 297.00000000000006
22.172405686131324 248.9999999999999
22.36160224663487 197.99999999999997
22

22.960905740798243 426.9999999999999
22.7945010619125 299.9999999999999
22.914542980203496 244.99999999999991
23.41251810818783 125.99999999999999
23.38210005838767 84.00000000000001
19.33828182790834 31.0
19.43924408912782 8.000000000000002
19.594655838634385 1.0
19.52653874893243 3.0
19.629264746778517 8.000000000000002
19.851482324038223 41.00000000000001
20.021729159457198 118.00000000000003
20.24817434695976 380.0000000000001
20.477058792553517 724.0
20.65241329429675 333.99999999999994
20.943132856054206 154.00000000000003
21.16804941435376 173.0
21.34904567167281 226.00000000000006
21.63742220825728 253.99999999999991
21.82225077023228 204.00000000000006
22.008588408506135 270.00000000000006
22.196447444287497 357.9999999999999
22.130740778923823 600.9999999999999
22.319597156485475 546.0
22.509995496394392 433.0000000000001
22.701948388365114 257.0
22.895468524903855 206.99999999999997
22.97786809281478 105.99999999999996
23.223640635968692 63.99999999999998
19.091615813426383 

In [181]:

coefs = pd.Series(lngr.coef_, index = X_train.columns)
coefs

#imp_coefs = pd.concat([coefs.sort_values().head(10),
#                     coefs.sort_values().tail(10)])
#imp_coefs.plot(kind = "barh", color='yellowgreen')
#plt.title("Features importance in the Lasso Model")
#plt.show()

season      0.173096
temp        0.047093
humidity   -0.015497
hour        0.097579
dtype: float64

In [144]:
parameters = {'alpha':[10**(-x) for x in range(0,10)]}
lassogs = GridSearchCV(Lasso(),param_grid=parameters,scoring='neg_mean_squared_error',cv=5);
X_train_scaled = RobustScaler().fit_transform(X_train)
#lasso = make_pipeline(RobustScaler(),lassogs)
lassogs.fit(X_train_scaled,Y_train);

print("The best value of Alpha is: ",lassogs.best_params_)
#
print("The best score achieved with Alpha=14 is: ",lassogs.best_score_)
#lasso



GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06,
                                   1e-07, 1e-08, 1e-09]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

The best value of Alpha is:  {'alpha': 0.001}
The best score achieved with Alpha=14 is:  -1.052632598094904


In [137]:
rel1 = rms_ev(5,lasso)

In [121]:
rel1.mean()
rel1.std()

1.4474764949245877

0.0888200962270124